In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint

from scipy import stats
from quantopian.pipeline.factors import AnnualizedVolatility, Factor, Returns, CustomFactor
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.filters import  StaticAssets
from quantopian.pipeline.filters.morningstar import Q500US, Q1500US, Q3000US
from quantopian.pipeline.data.factset import EquityMetadata
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

In [ ]:
'''The rules for the strategy are:
1. Select all stocks near the market open whose returns from their previous day’s lows to today’s opens are lower 
   than one standard deviation. The standard deviation is computed using the daily close-to-close returns of the 
   last 90 days. These are the stocks that “gapped down.”
2. Narrow down this list of stocks by requiring their open prices to be higher than the 20-day moving average of 
   the closing prices.
3. Buy the 10 stocks within this list that have the lowest returns from their previous day’s lows. If the list has 
   fewer than 10 stocks, then buy the entire list.
4. Liquidate all positions at the market close.'''

class CloseOnN(CustomFactor):  
    # Define inputs
    inputs = [USEquityPricing.close]
    
    # Set window_length to one more  than the number of days to look back
    # my_close_on_10 = CloseOnN(window_length = 10+1)
    
    #window_length = 2 
    
    def compute(self, today, assets, out, close):  
        out[:] = close[0]
        
class LowOnN(CustomFactor):  
    # Define inputs
    inputs = [USEquityPricing.low]
    
    # Set window_length to one more  than the number of days to look back
    # my_close_on_10 = CloseOnN(window_length = 10+1)
    
    #window_length = 2 
    
    def compute(self, today, assets, out, low):  
        out[:] = low[0]
        
class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 90
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0)


In [ ]:
'''The rules for the strategy are:
1. Select all stocks near the market open whose returns from their previous day’s lows to today’s opens are lower 
   than one standard deviation. The standard deviation is computed using the daily close-to-close returns of the 
   last 90 days. These are the stocks that “gapped down.”
2. Narrow down this list of stocks by requiring their open prices to be higher than the 20-day moving average of 
   the closing prices.
3. Buy the 10 stocks within this list that have the lowest returns from their previous day’s lows. If the list has 
   fewer than 10 stocks, then buy the entire list.
4. Liquidate all positions at the market close.'''

opening = USEquityPricing.open.latest
close = USEquityPricing.close.latest
prevclose = CloseOnN(window_length = 2)
std_90 = Volatility(window_length=90)
sma_20 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=20)
low = USEquityPricing.low.latest
prevlow = LowOnN(window_length = 2)
ret = opening/prevlow-1

# Construct Filter.
rule_1 = ((opening/prevlow-1) < std_90)
rule_2 = (opening > sma_20)

# Create pipeline
universe = Q1500US
#universe = StaticAssets(symbols(['TSLA', 'AAPL']))

pipe = Pipeline(
        columns = {
        'open' : opening,
        'close' : close,
        'prevlow' : prevlow,
        'return' : ret,
        'rule_1' : rule_1,
        'rule_2' : rule_2})

pipe.set_screen (rule_1 & rule_2)

In [ ]:
df = run_pipeline(pipe, '2019-08-01', '2020-08-01')
df

In [ ]:
df = df.sort_values(by = 'return', ascending = True)

In [ ]:
df

In [ ]:
len(df.index.get_level_values(1))

In [ ]:
np.diff()